# Entrainment <a class='tocSkip'></a>

Thomas Schanzer z5310829  
School of Physics, UNSW  
September 2021

In this notebook, we seek to determine the effect of entrainment on a descending parcel.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Theory" data-toc-modified-id="Theory-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Theory</a></span><ul class="toc-item"><li><span><a href="#Step-1:-mixing-without-phase-change" data-toc-modified-id="Step-1:-mixing-without-phase-change-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Step 1: mixing without phase change</a></span></li><li><span><a href="#Step-2:-reaching-phase-equilibrium" data-toc-modified-id="Step-2:-reaching-phase-equilibrium-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Step 2: reaching phase equilibrium</a></span></li><li><span><a href="#Step-3:-descent" data-toc-modified-id="Step-3:-descent-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Step 3: descent</a></span></li></ul></li></ul></div>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import metpy.calc as mpcalc
import metpy.constants as const
from metpy.units import units
from metpy.units import concatenate

from scipy.optimize import root_scalar

from os import mkdir
from os.path import exists
import sys

## Theory

We imagine a parcel, whose properties are denoted by a subscript P, and its environment, whose properties are denoted by a subscript E. As a result of descending through a distance $dz$, equal masses $dm$ of parcel and environmental air have been exchanged:
<img src='diagram1.png' style="width: 450px;">

The size of $dm$ is related to $dz$ by the *entrainment rate* $\lambda$, which is defined as the fraction of the parcel's mass that is exchanged per unit height:
$$ \lambda \equiv \frac{d\left(\frac{m}{m_P}\right)}{dz}. $$
Thus $dm = \lambda m_P dz$.
We now consider the parcel as a container with an imaginary partition between the original portion with mass $m_P - dm$ and the entrained portion with mass $dm$:
<img src='diagram2.png' style="width: 450px;">

The partition is then removed, and a three-step process occurs:
1. The two parts mix, conserving enthalpy and water mass without any phase changes,
2. The liquid and vapour phases of water come into equilibrium, with phase changes if necessary, and
3. The parcel descends a distance $dz$, either dry or moist adiabatically.

### Step 1: mixing without phase change
Enthalpy conservation requires that
$$
\begin{align}
    (m_P - dm) c_p T_P + dm~c_p T_E &= m_P c_p T' \\
    \Leftrightarrow \qquad T' &= \frac{(m_P - dm) c_p T_P + dm~c_p T_E}{m_P} \\
    &= T_P + \frac{dm}{m_P} (T_E - T_P) \\
    &= T_P + \lambda (T_E - T_P) dz.
\end{align}
$$
Conservation of water vapour mass (with no condensation) requires that
$$
\begin{align}
    m_{v,P} + m_{v,E} &= m_v' \\
    \Leftrightarrow \qquad (m - dm) q_P + dm~q_E &= m_P q' \\
    \Leftrightarrow \qquad q' &= q_P + \lambda (q_E - q_P) dz.
\end{align}
$$
If we use $l$ to denote a ratio of liquid mass to total mass, conservation of liquid water mass gives an identical relation
$$ l' = l_P + \lambda (l_E - l_P) dz. $$

In [1]:
def mix(parcel, environment, rate, dz):
    return parcel + rate * (environment - parcel) * dz

### Step 2: reaching phase equilibrium

Let us denote the saturation specific humidity at pressure $p$ and temperature $T$ by $q^*(p,T)$. We wish to find the final specific humidity $q''$, liquid ratio $l''$ and temperature $T''$ of the parcel once phase equilibrium is reached.

To start with, we know that any phase changes must conserve total water mass:
$$ q' + l' = q'' + l''.$$

We also know that the heat $dQ$ required to increase the specific humidity by $dq$ is given by
$$ \frac{dQ}{m} = -L~dq = L(q' - q'') = L(l'' - l'), $$
and if the process occurs at constant pressure, the corresponding change in temperature will be
$$
\begin{align}
    c_p~dT &= -L~dq \\
    \Leftrightarrow \qquad T'' &= T' + \frac{L(q' - q'')}{c_p} = T' + \frac{L(l'' - l')}{c_p}.
\end{align}
$$

**Case 1:** If $q' > q^*(p,T')$, water vapour must condense and the parcel will warm until it reaches a state where
$$
\begin{align}
    q'' &= q^*(p,T'') \\
    &= q^* \left(p, T' + \frac{L(q' - q'')}{c_p} \right),
\end{align}
$$
with $l'' = l' + q' - q''$.

**Case 2:** If $q' \le q^*(p,T')$ and $l' > 0$, liquid water must evaporate and the parcel will cool until either
- **Case 2.1:** all liquid has evaporated, i.e., $l'' = 0$ and 
$$
    q'' = q' + l' < q^*(p,T'') = q^* \left(p, T' - \frac{Ll'}{c_p} \right)
$$
    or
- **Case 2.2:** the parcel reaches saturation, i.e.,
$$
\begin{align}
    q'' &= q^*(p,T'') \\
    &= q^* \left(p, T' + \frac{L(q' - q'')}{c_p} \right),
\end{align}
$$
    with $l'' = l' + q' - q''$. This is identical to Case 1, except that here $q'' > q'$.
    
The equations for all cases can be solved numerically.

In [7]:
def temperature_change(dq):
    return -const.water_heat_vaporization * dq / const.dry_air_spec_heat_press

def q_root(q_final, pressure, t_initial, q_initial):
    q_final = q_final*units.dimensionless
    t_final = t_initial + temperature_change(q_final - q_initial)
    q_final_saturated = mpcalc.specific_humidity_from_mixing_ratio(
        mpcalc.saturation_mixing_ratio(pressure, t_final))
    return (q_final - q_final_saturated).to(units.dimensionless).m

def equilibrium(
        pressure, t_parcel, q_parcel, l_parcel, t_env, q_env, l_env, rate, dz):
    # mixing without phase change
    t_mixed = mix(t_parcel, t_env, rate, dz)
    q_mixed = mix(q_parcel, q_env, rate, dz)
    l_mixed = mix(l_parcel, l_env, rate, dz)
    q_mixed_saturated = mpcalc.specific_humidity_from_mixing_ratio(
        mpcalc.saturation_mixing_ratio(pressure, t_mixed))
    
    if q_mixed > q_mixed_saturated:
        # we need to condense water vapour
        sol = root_scalar(
            q_root, args=(pressure, t_mixed, q_mixed), bracket=[0, q_mixed])
        q_final = sol.root
        t_final = t_mixed + temperature_change(q_final - q_mixed)
        l_final = l_mixed + q_mixed - q_final
        return (t_final, q_final, l_final)
    elif q_mixed <= q_mixed_saturated and l_mixed > 0:
        # we need to evaporate liquid water
        
        # if all liquid evaporates:
        t_all_evap = t_mixed + temperature_change(l_mixed)
        q_all_evap_saturated = mpcalc.specific_humidity_from_mixing_ratio(
        mpcalc.saturation_mixing_ratio(pressure, t_all_evap))
        
        if q_mixed + l_mixed < q_all_evap_saturated:
            return (t_all_evap, q_all_evap, 0*units.dimensionless)
        else:
            sol = root_scalar(
                q_root, args=(pressure, t_mixed, q_mixed),
                bracket=[q_mixed, q_mixed + l_mixed])
            q_final = sol.root
            t_final = t_mixed + temperature_change(q_final - q_mixed)
            l_final = l_mixed + q_mixed - q_final
            return (t_final, q_final, l_final)
    else:
        # already in equilibrium, no action needed
        return (t_mixed, q_mixed, l_mixed)

### Step 3: descent

Once the temperature, specific humidity and liquid content of the parcel after mixing are known, it is a relatively simple matter to use the functions of MetPy and those we have already written to determine how they change when the parcel descends a distance $dz$.